In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt


## Runoff historic stats

In [3]:
MAR = xr.open_dataset(r'\MAR.nc', engine='netcdf4')
summed_sector_discharge = MAR.sum(dim='station')

In [ ]:
summed_sector_discharge_df = pd.DataFrame(data=summed_sector_discharge['discharge'].values, index=MAR.time, columns=['summed_discharge'])
mask = (summed_sector_discharge_df.index.month == 8) & (summed_sector_discharge_df.index.day >= 28) | (summed_sector_discharge_df.index.month == 9) & (summed_sector_discharge_df.index.day <= 30) # late season

late_season = summed_sector_discharge_df[mask]
print(late_season.max())

top_10_largest = late_season.nlargest(10, 'summed_discharge')
print(top_10_largest)

In [2]:
MAR = xr.open_dataset(r'\MAR.nc', engine='netcdf4')

land_term_stations = [66855,66850,66847,66834,66819,66783,66768,66755,66595,66592,66591,66576,66574,66571,66564,66561,66554,66549,66541,66533,66532,66530,66518,66517,66480,66474,66472,66463,66451,66446,66445,66444,66416,66414,66407,66388,66356,66353,66350,66346,66344] # land-terminating outlets downstream of flux gates
station_sel = MAR.sel(station=land_term_stations)
summed_sector_discharge = station_sel.sum(dim='station')

In [ ]:
summed_sector_discharge_df = pd.DataFrame(data=summed_sector_discharge['discharge'].values, index=MAR.time, columns=['summed_discharge'])
mask = (summed_sector_discharge_df.index.month == 8) & (summed_sector_discharge_df.index.day >= 15) | (summed_sector_discharge_df.index.month == 9) & (summed_sector_discharge_df.index.day <= 30) # late season

late_season = summed_sector_discharge_df[mask]
print(late_season.max())

top_10_largest = late_season.nlargest(10, 'summed_discharge')
print(top_10_largest)

In [ ]:
melt_variation = late_season.diff()
print(melt_variation)

top_10_largest_diff = melt_variation.nlargest(10, 'summed_discharge')
print(top_10_largest_diff)

In [ ]:
melt_variation_all = summed_sector_discharge_df.diff()
print(melt_variation_all)

top_10_largest_diff2 = melt_variation_all.nlargest(10, 'summed_discharge')
print(top_10_largest_diff2)

In [ ]:
summed_sector_discharge.loc[(summed_sector_discharge['time'].dt.day==25) & (summed_sector_discharge['time'].dt.month==[8,9])]


# Land-terminating Sector

In [7]:
MAR = xr.open_dataset(r'\data\RUNOFF\MAR.nc', engine='netcdf4')
MAR_22 =  MAR.sel(time=slice('2022-01','2022-12')) # 2022


land_term_stations = [66855,66850,66847,66834,66819,66783,66768,66755,66595,66592,66591,66576,66574,66571,66564,66561,66554,66549,66541,66533,66532,66530,66518,66517,66480,66474,66472,66463,66451,66446,66445,66444,66416,66414,66407,66388,66356,66353,66350,66346,66344] # land-terminating outlets downstream of flux gates
station_sel = MAR_22.sel(station=land_term_stations)
summed_sector_discharge = station_sel.sum(dim='station')

In [ ]:
sector_discharge = pd.DataFrame(data=summed_sector_discharge['discharge'].values, index=MAR_22.time, columns=['summed_discharge'])
sector_discharge = sector_discharge[(sector_discharge.index >= '2022-04-01') & (sector_discharge.index <= '2022-11-30')] # subsetting to melt-season

### removing timestamps with high melt in Septemer ################
sector_discharge_nomelt = sector_discharge[(sector_discharge.index < '2022-09-02') | (sector_discharge.index > '2022-09-05')] # first melt event
sector_discharge_nomelt = sector_discharge_nomelt[(sector_discharge_nomelt.index < '2022-09-09') | (sector_discharge_nomelt.index > '2022-09-11')] #2nd melt event.....
sector_discharge_nomelt = sector_discharge_nomelt[(sector_discharge_nomelt.index < '2022-09-13') | (sector_discharge_nomelt.index > '2022-09-20')]
sector_discharge_nomelt = sector_discharge_nomelt[(sector_discharge_nomelt.index < '2022-09-23') | (sector_discharge_nomelt.index > '2022-09-24')]
sector_discharge_nomelt = sector_discharge_nomelt[(sector_discharge_nomelt.index < '2022-09-26') | (sector_discharge_nomelt.index > '2022-09-27')]
sector_discharge_nomelt = sector_discharge_nomelt.resample('D').interpolate() # linearly interpolating over removed melt events

#### plots with and without melt events ############################
sector_discharge.plot() # plot with melt events
sector_discharge_nomelt.plot()  # plot with melt events removed

######### calculating total discharge ################################
total_discharge = sector_discharge.sum() * 86400 # convert to m3
total_discharge = ((total_discharge * 1000)/1000000000000) * (1000/917) # convert to Gt of ice

total_discharge_nomelt = sector_discharge_nomelt.sum() * 86400 # convert to m3
total_discharge_nomelt = ((total_discharge_nomelt * 1000)/1000000000000) * (1000/917) # convert to Gt of ice

percent_increase = ((total_discharge - total_discharge_nomelt)/ total_discharge_nomelt) * 100   # percentage increase in discharge

print('Annual discharge: %.2f Gt \nAnnual discharge (nomelt): %.2f Gt \nPercentage increase: %.2f %%' % (total_discharge, total_discharge_nomelt, percent_increase))


Timesteps to remove: 2nd, 3rd, 4th, 5th    17th, 18th     23rd, 24th     26th, 27th

# Marine-terminating sector

In [24]:
MAR = xr.open_dataset(r'\data\RUNOFF\MAR.nc', engine='netcdf4')
MAR_22 =  MAR.sel(time=slice('2022-01','2022-12')) # 2022

mtg_stations = [73658,73656,73652,73648,73622,73601,73562]
station_sel = MAR_22.sel(station=mtg_stations)
summed_sector_discharge_mtg = station_sel.sum(dim='station')

In [ ]:
sector_discharge = pd.DataFrame(data=summed_sector_discharge_mtg['discharge'].values, index=MAR_22.time, columns=['summed_discharge'])
sector_discharge = sector_discharge[(sector_discharge.index >= '2022-04-01') & (sector_discharge.index <= '2022-11-30')] # subsetting to melt-season

### removing timestamps with high melt in Septemer ################
sector_discharge_nomelt = sector_discharge[(sector_discharge.index < '2022-09-02') | (sector_discharge.index > '2022-09-05')] # first melt event
sector_discharge_nomelt = sector_discharge_nomelt[(sector_discharge_nomelt.index < '2022-09-09') | (sector_discharge_nomelt.index > '2022-09-11')] #2nd melt event.....
sector_discharge_nomelt = sector_discharge_nomelt[(sector_discharge_nomelt.index < '2022-09-13') | (sector_discharge_nomelt.index > '2022-09-20')]
sector_discharge_nomelt = sector_discharge_nomelt[(sector_discharge_nomelt.index < '2022-09-23') | (sector_discharge_nomelt.index > '2022-09-24')]
sector_discharge_nomelt = sector_discharge_nomelt[(sector_discharge_nomelt.index < '2022-09-26') | (sector_discharge_nomelt.index > '2022-09-27')]
sector_discharge_nomelt = sector_discharge_nomelt.resample('D').interpolate() # linearly interpolating over removed melt events

#### plots with and without melt events ############################
sector_discharge.plot() # plot with melt events
sector_discharge_nomelt.plot()  # plot with melt events removed

######### calculating total discharge ################################
total_discharge = sector_discharge.sum() * 86400 # convert to m3
total_discharge = ((total_discharge * 1000)/1000000000000) * (1000/917) # convert to Gt of ice

total_discharge_nomelt = sector_discharge_nomelt.sum() * 86400 # convert to m3
total_discharge_nomelt = ((total_discharge_nomelt * 1000)/1000000000000) * (1000/917) # convert to Gt of ice

percent_increase = ((total_discharge - total_discharge_nomelt)/ total_discharge_nomelt) * 100   # percentage increase in discharge

print('Annual discharge: %.2f Gt \nAnnual discharge (nomelt): %.2f Gt \nPercentage increase: %.2f %%' % (total_discharge, total_discharge_nomelt, percent_increase))